In [ ]:
import subprocess
subprocess.check_call(["pip", "install", "selenium"])
subprocess.check_call(["pip", "install", "webdriver-manager"])
subprocess.check_call(["pip", "install", "requests"])

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
import pandas as pd
import requests

In [7]:
def abrir_navegador():
    """
    Abre o navegador e acessa o site do IBGE.

    :return: Navegador aberto.
    """
    opcoes = Options()
    # opcoes.add_argument('--headless') # Rodar sem abrir o navegador
    opcoes.add_argument('--auto-open-devtools-for-tabs') # Abrir o console do navegador
    opcoes.add_argument('--start-maximized')
    navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=opcoes)
    return navegador

def fechar_navegador(navegador):
    """
    Fecha o navegador.

    :param navegador: Navegador aberto.
    """
    navegador.quit()

def retornar_tabela():
    """
    Retorna a tabela com os dados de população por município do estado de São Paulo.

    :return: Tabela com os dados do site do IBGE.
    """
    elemento = (By.XPATH, '//*[@id="municipios"]')
    tabela = WebDriverWait(navegador, 20).until(EC.presence_of_element_located(elemento))
    return tabela

def acessar_site(url):
    """
    Acessa o site pela URL.

    :param navegador: Navegador aberto.
    :param url: URL do site do IBGE.
    """
    navegador.get(url)
    tabela = retornar_tabela()
    return tabela


In [27]:
def dados_populacao():
    """
    Retorna um data frame com os dados de população por município do estado de São Paulo.

    :return: Data frame com os dados de população por município do estado de São Paulo.
    """
    tabela = acessar_site('https://cidades.ibge.gov.br/brasil/sintese/sp?indicadores=96385,96386,29765,29763,60036,47001')
    df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
    df.drop(columns=['Gentílico'], inplace=True)
    df.drop(df.index[-1], inplace=True)
    df.rename(columns={'Municípios': 'cidade',
                       'População no último censo': 'populacao_censo',
                       'Pessoal ocupado': 'populacao_ocupado',
                       'População ocupada': 'porcentagem_populacao_ocupado',
                       'Salário médio mensal dos trabalhadores formais': 'salario_medio_mensal_trabalhadores_formais',
                       'PIB per capita': 'pib_per_capita',
                       'Densidade demográfica': 'densidade_demografica'}, inplace=True)
    df['cidade'] = df['cidade'].astype(str)
    df['populacao_censo'] = df['populacao_censo'].str.replace(' pessoas', '').astype(int)
    df['populacao_ocupado'] = df['populacao_ocupado'].str.replace(' pessoas', '').astype(int)
    df['porcentagem_populacao_ocupado'] = df['porcentagem_populacao_ocupado'].str.replace(' %', '').str.replace(',', '.').astype(float)
    df['salario_medio_mensal_trabalhadores_formais'] = df['salario_medio_mensal_trabalhadores_formais'].str.replace(' salários mínimos', '').str.replace(',', '.').astype(float)
    df['pib_per_capita'] = df['pib_per_capita'].str.replace(' R$', '').str.replace(',', '.').astype(float)
    df['densidade_demografica'] = df['densidade_demografica'].str.replace(' habitante por quilômetro quadrado', '').str.replace(',', '.').astype(float)
    return df

def dados_cidade():
    """
    Retorna um data frame com os dados referêntes a municípios do estado de São Paulo.

    :return: Data frame com os dados de população por município do estado de São Paulo.
    """
    tabela = acessar_site('https://cidades.ibge.gov.br/brasil/sintese/sp?indicadores=30255,95335,60029,60031,29167')
    df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
    df.drop(columns=['Gentílico'], inplace=True)
    df.drop(df.index[-1], inplace=True)
    df.rename(columns={'Municípios': 'cidade',
                       'Área da unidade territorial': 'area_territorio',
                       'Área urbanizada': 'area_urbanizada',
                       'Urbanização de vias públicas': 'urbanizacao_vias_publicas',
                       'Arborização de vias públicas': 'arborizacao_vias_publicas',
                       'Índice de Desenvolvimento Humano Municipal (IDHM)': 'idhm'}, inplace=True)
    df['cidade'] = df['cidade'].astype(str)
    df['area_territorio'] = df['area_territorio'].str.replace(' km²', '').str.replace(',', '.').astype(float)
    df['area_urbanizada'] = df['area_urbanizada'].str.replace(' km²', '').str.replace(',', '.').astype(float)
    df['urbanizacao_vias_publicas'] = df['urbanizacao_vias_publicas'].str.replace(' %', '').str.replace(',', '.').astype(float)
    df['arborizacao_vias_publicas'] = df['arborizacao_vias_publicas'].str.replace(' %', '').str.replace(',', '.').astype(float)
    df['idhm'] = df['idhm'].str.replace(',', '.').astype(float)
    return df

In [28]:
navegador = abrir_navegador()
df_populacao = dados_populacao()
df_cidade = dados_cidade()
fechar_navegador(navegador)


# unir os dois data frames por cidade
df = pd.merge(df_populacao, df_cidade, on='cidade')
df.head(1000)

C:\Users\ander\AppData\Local\Temp\ipykernel_4960\2975070796.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
C:\Users\ander\AppData\Local\Temp\ipykernel_4960\2975070796.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]


,cidade,populacao_ocupado,salario_medio_mensal_trabalhadores_formais,pib_per_capita,porcentagem_populacao_ocupado,populacao_censo,densidade_demografica,area_territorio,idhm,arborizacao_vias_publicas,urbanizacao_vias_publicas,area_urbanizada
0,Adamantina,12303,2.2,35788.26,33.7,34687,84.19,411.987,790.0,99.3,17.9,12.43
1,Adolfo,738,1.8,38464.09,22.2,4351,20.62,211.055,730.0,98.6,4.9,2.33
2,Aguaí,7429,2.3,31446.28,18.7,32072,67.58,474.554,715.0,91.1,14.7,6.94
3,Águas da Prata,1167,2.2,22718.42,13.8,7369,51.65,142.673,781.0,81.4,46.1,3.27
4,Águas de Lindóia,6138,1.9,26418.38,30.5,17930,298.21,60.126,745.0,82.1,73.3,8.53
...,...,...,...,...,...,...,...,...,...,...,...,...
640,Vista Alegre do Alto,3367,2.9,44966.33,35.7,8109,84.97,95.429,744.0,98.1,44.2,2.03
641,Vitória Brasil,247,2.2,23045.31,14.5,1794,36.00,49.832,725.0,99.6,5.8,0.51
642,Votorantim,23262,2.6,27713.57,18.3,127923,694.53,184.186,767.0,83.4,58.2,25.54
643,Votuporanga,29941,2.2,35390.26,31.3,96634,229.70,420.703,790.0,97.2,50.8,25.65


In [98]:
url = "https://servicodados.ibge.gov.br/api/v1/localidades/estados/SP/municipios/"

try:
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        df = pd.json_normalize(data)
        df = df[['nome', 'microrregiao.nome', 'microrregiao.mesorregiao.nome', 'regiao-imediata.nome', 'regiao-imediata.regiao-intermediaria.nome']]
        df.rename(columns={
            'nome': 'cidade',
            'microrregiao.nome': 'microrregiao',
            'microrregiao.mesorregiao.nome': 'mesorregiao',
            'regiao-imediata.nome': 'regiao_imediata',
            'regiao-imediata.regiao-intermediaria.nome': 'regiao_intermediaria'}, inplace=True)
    else:
        print(f"A solicitação falhou com status {response.status_code}")
except Exception as e:
    print(f"Ocorreu um erro durante a solicitação: {e}")

df.head()
# incompleto

,cidade,microrregiao,mesorregiao,regiao_imediata,regiao_intermediaria
0,Adamantina,Adamantina,Presidente Prudente,Adamantina - Lucélia,Presidente Prudente
1,Adolfo,São José do Rio Preto,São José do Rio Preto,São José do Rio Preto,São José do Rio Preto
2,Aguaí,Pirassununga,Campinas,São João da Boa Vista,Campinas
3,Águas da Prata,São João da Boa Vista,Campinas,São João da Boa Vista,Campinas
4,Águas de Lindóia,Amparo,Campinas,Amparo,Campinas
